<a href="https://colab.research.google.com/github/HectorGtz27/FlameForeCastModel/blob/main/ModeloEntrenado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importar librerias

1.Preprocesamiento de Datos: Estandariza o normaliza las variables para que todas estén en la misma escala. Esto es especialmente importante para redes neuronales.

In [60]:
# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

data_with_coordinates = {
    'Lat': [34.0522, 34.0523, 34.0524, 34.0525, 34.0526],
    'Long': [-118.2437, -118.2438, -118.2439, -118.2440, -118.2441],
    'TMP': [25, 30, 27, 22, 29],
    'RH': [40, 35, 50, 60, 38],
    'WS': [10, 12, 8, 6, 11],
    'WD': [180, 200, 190, 170, 185],
    'FFMC': [85, 88, 80, 75, 87],
    'DMC': [30, 25, 40, 45, 28],
    'DC': [700, 690, 750, 770, 680],
    'ISI': [8, 9, 6, 5, 10],
    'BUI': [40, 35, 50, 55, 37],
    'FWI': [20, 22, 15, 10, 25]
}

df = pd.DataFrame(data_with_coordinates)

# Estandarizar las variables
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)




2.División del Conjunto de Datos: Divide los datos en conjuntos de entrenamiento, validación y prueba.

In [61]:
from sklearn.model_selection import train_test_split

X = df_scaled.drop('FWI', axis=1)
y = df_scaled['FWI']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape para CNN y LSTM
X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)

# Añadir una dimensión de etapa de tiempo
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1], 1))





Reshape para CNN y LSTM

Paso 3. Arquitectura de la Red: Define la arquitectura de tu red. Esto incluye elegir el número de capas ocultas y neuronas en cada capa. También debes decidir qué funciones de activación usar.

In [62]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, LSTM, TimeDistributed

model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, X_train.shape[2], 1)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))




Paso 4. Eleccion del modelo

Las redes neuronales convolucionales (CNN) se utilizan comúnmente para el análisis de imágenes, pero también pueden aplicarse a datos secuenciales o geoespaciales. Podríamos diseñar una CNN para procesar nuestros datos, que incluyen tanto mediciones meteorológicas como coordenadas geográficas.

Incorporar unidades de memoria a largo plazo (Long Short-Term Memory, LSTM) es una excelente idea si estás trabajando con datos que tienen componentes secuenciales o temporales, como en el caso de datos meteorológicos y de incendios a lo largo del tiempo. Los LSTM son útiles para capturar patrones a largo plazo en series temporales, lo que podría ser fundamental para predecir la propagación de incendios forestales basándose en condiciones que han sido observadas en años anteriores.

Podríamos considerar una arquitectura híbrida que combine Convolutional Neural Networks (CNN) para capturar patrones espaciales y LSTM para patrones temporales.



In [63]:
# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')





 Paso 5. Evaluación del modelo.

 En esta fase, vamos a utilizar nuestro conjunto de datos de prueba para evaluar el rendimiento del modelo. Queremos asegurarnos de que nuestro modelo no solo se ajusta bien a los datos de entrenamiento, sino que también generaliza bien a nuevos datos.

In [65]:
# Entrenamiento
model.fit(X_train, y_train, epochs=10, verbose=1)

# Evaluación del modelo en el conjunto de prueba
loss = model.evaluate(X_test, y_test, verbose=1)

print("Pérdida del modelo en el conjunto de prueba: {}".format(loss))

# Si deseas realizar más evaluaciones, como calcular métricas adicionales, aquí es donde lo harías.



Epoch 1/10
1/1 [==============================] - 0s 15ms/step - loss: 0.1391
Epoch 2/10
1/1 [==============================] - 0s 14ms/step - loss: 0.1375
Epoch 3/10
1/1 [==============================] - 0s 13ms/step - loss: 0.1479
Epoch 4/10
1/1 [==============================] - 0s 13ms/step - loss: 0.1496
Epoch 5/10
1/1 [==============================] - 0s 15ms/step - loss: 0.1334
Epoch 6/10
1/1 [==============================] - 0s 13ms/step - loss: 0.1029
Epoch 7/10
1/1 [==============================] - 0s 17ms/step - loss: 0.0675
Epoch 8/10
1/1 [==============================] - 0s 13ms/step - loss: 0.0368
Epoch 9/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0170
Epoch 10/10
1/1 [==============================] - 0s 268ms/step - loss: 0.3388
Pérdida del modelo en el conjunto de prueba: 0.3387514352798462


Paso 6: Evaluación del Modelo

In [66]:
# Evaluación del modelo en el conjunto de prueba
test_loss = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss}")

# Si quieres también puedes calcular otras métricas como R² aquí


1/1 [==============================] - 0s 26ms/step - loss: 0.3388
Test Loss: 0.3387514352798462


Paso 7: Interpretación de Resultados

In [67]:
# Puedes acceder a los pesos así
for layer in model.layers:
    print(layer.get_weights())


[array([[[-0.00335486, -0.23355787, -0.04982813, -0.11042888,
          0.13263391,  0.2037471 ,  0.26635727, -0.16483916,
          0.25446764,  0.10234929,  0.2768368 ,  0.16859141,
         -0.18510026, -0.3088163 ,  0.09389178,  0.0973658 ,
          0.31191233, -0.23329757,  0.15030271, -0.24696322,
         -0.25880787,  0.29749107, -0.2670706 , -0.10080856,
          0.16318582,  0.08785996,  0.07251623, -0.17509902,
          0.27261463, -0.19241534,  0.27429864, -0.20034978,
         -0.02844257,  0.3040553 , -0.31043828, -0.09104276,
         -0.29037857, -0.26598674,  0.19546787, -0.11384574,
         -0.02107853,  0.06900523, -0.03302093, -0.10385247,
          0.08613374, -0.25293982,  0.05599487,  0.22942163,
          0.05246976,  0.151386  , -0.07062382, -0.20505498,
         -0.1091615 ,  0.1822259 , -0.04326859,  0.02431041,
         -0.19114216,  0.14140554,  0.06494162, -0.09873202,
          0.07613515,  0.2702101 ,  0.05396478, -0.29154274]]],
      dtype=float32)

Paso 8: Puesta en Producción
Finalmente, una vez que estás satisfecho con tu modelo, puedes proceder a implementarlo en un entorno de producción. Esto podría implicar guardar el modelo entrenado y luego cargarlo en un servicio que pueda acceder a él para realizar predicciones en tiempo real.

In [70]:
# Para guardar el modelo
model.save('my_model.h5')

# Para cargar el modelo
from tensorflow.keras.models import load_model
new_model = load_model('my_model.h5')
